# DATA604 Assignment 1
### Alexander Simon
### 2024-06-07

# 1. Submit a document verifying that you have successfully installed and tested Simio and Python

## 1.1. Simio
As I noted in class, the download link provided in the course materials installed a 30-day trial version of the full program. I’m not sure whether it will default to the Personal Edition after this time, but I’ll see what happens after the trial ends.
![Simio welcome.png](https://github.com/alexandersimon1/Data604/blob/main/Assignment1/Simio%20welcome.png?raw=true)

## 1.2. Python
I already had Python installed on my computer, so below I just show that it works.

In [7]:
print("Hello world!")

Hello world!


# 2. RealPython SimPy tutorial
I installed simpy, downloaded the tutorial script (see code block below), and followed the tutorial.

## 2.1. Minimum wait time
The shortest wait time I got was 3 minutes 25 seconds with 12 cashiers, 10 servers, and 2 ushers.

## 2.2. Minimum number of employees
The solution above requires 24 employees. Although not specified in the tutorial, the manager will also need to consider how much it costs to pay all the employees, so the problem could be rephrased as minimizing the total number of employees while keeping the wait time to 10 minutes or less. With this additional constraint, the best solution I found was 9 cashiers, 6 servers, and 1 usher (16 employees total). This assumes that all employees are paid the same.<br>
<br>
I found these solutions by trial and error. It would be better to determine the optimal soluton programmatically, but I don't know how to do that yet.

In [6]:
"""Companion code to https://realpython.com/simulation-with-simpy/

'Simulating Real-World Processes With SimPy'

Python version: 3.7.3
SimPy version: 3.0.11
"""

import random
import statistics
import simpy

wait_times = []


class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)

    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.randint(1, 3))

    def check_ticket(self, moviegoer):
        yield self.env.timeout(3 / 60)

    def sell_food(self, moviegoer):
        yield self.env.timeout(random.randint(1, 5))


def go_to_movies(env, moviegoer, theater):
    # Moviegoer arrives at the theater
    arrival_time = env.now

    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))

    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))

    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))

    # Moviegoer heads into the theater
    wait_times.append(env.now - arrival_time)


def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)

    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))


def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)


def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. Simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params


def main():
    # Setup
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
    )


if __name__ == "__main__":
    main()

Running simulation... 
The average wait time is 3 minutes and 25 seconds.


# 3. Select one case from https://www.simio.com/academics/student-projects/ and analyze how Simio helped improve the process in the case study

Selected case: [Improving Disney World’s monorail system](https://www.simio.com/academics/student-projects/transportation/20131227-Improving-Disney-Worlds-Monorail-System/)
<br><br>
Due to the design of the monorail stations at Walt Disney World, guests in wheelchairs need extra time and staff assistance to safely cross a gap between the station platform and the train, which delays the train by 5 minutes. Two potential solutions are to increase the number of trains or modify the platforms to eliminate the gap so that all passengers can board without assistance. Simio enabled comparing these scenarios under different conditions (eg, number of stations, trains, and passengers). The best performing model was platform modfication, which maximized customer accessibility and enabled all trains to stop at each station for only 3 minutes.

# 4. Write a short essay explaining which other factors could be considered to build a stronger simulation model, and how you can apply simulation and modeling in your professional duties.

The model for the Disney monorail system could be improved by identifying the platforms that would benefit most from modification, cost of the modifications, and the optimal sequence of performing the construction to minimize service disruption. The first two are important considerations because the cost of modifying all platforms may be substantial. If some stations are more affected by wheelchair accessibility delays than others, then cost savings may be possible by limiting modifications to those stations.<br>
<br>
In addition, if the platform modification requires closing the station for a period of time while construction is performed, not all stations could be modified simultaneously (ie, people may not be happy about having to walk/wheel through the entire resort). Similarly, closing several adjacent stations at the same time may make it inconvenient for people to travel between attractions. So, it may be beneficial to know average walking/wheeling times between stations and to model different closure scenarios to determine the optimal order of construction. Of course, if the platform modification is a simple fix that could be performed while the resort is closed at night, then modeling would not be needed.<br>
<br>
Currently, I am a medical writer. In medical communications, generative AI (eg, “ChatGPT”) is a trending topic. One unresolved question is how writers can best use the tool to create scientific documents. On one hand, generative AI is very fast, so it could potentially increase a writer’s productivity. However, it also has a tendency of “hallucinating” (eg, quoting nonexistent sources as factual), which requires writers to spend time fact checking and correcting generated text, potentially negating any productivity gains from the speed of generation. The question of whether generative AI can increase productivity in medical writing could be explored with a model of productivity in a medical communications office (ie, time and cost for writers to create different types of documents) and using simulations to compare productivity with or without generative AI. It may be possible to adapt existing project management simulations, such as the PM game simulation shown in the Week 1 meetup, for this problem.
